# Project2
Новак Евгений <br>
Орлов Григорий <br>
Тожимухаммедов Асадбек <br>
Вариант: 1

----

## Условие: <br>
* **Реализовать метод наискорейшего спуска (золотое сечение), метод Левенберга-Марквартда**<br>

* **Тестовые функции**

1. Расчет минимума сильно-выпуклой функции:
		$$
		f(x) = \frac{L - \mu}{8} \left[ x_1^2 + \sum_{i=2}^n (x_i - x_{i+1})^2 - 2x_1 \right] + \frac{\mu}{2} \| x \|_2^2
		$$
		Полагаем $ L = 100, \mu = 0.1 $.

2. Расчет двойственной задачи к задаче расчета матрицы корреспонденций:
		$$
		f(x, y) = -(L, x) - (W, y) + \ln \left[ \sum_{i,j} \exp(-\alpha c_{ij} + x_i + y_j) \right]
		$$  
		Здесь: $ x \in R^n, \, y \in R^n, \, L, W \in R_{+}^n, \, \| L \|_1 = 1, \, \| W \|_1 = 1, \, \alpha \geq 1, \, c_{ij} \in [0, 1] $. Параметры $ L, W, c $ задаются случайно, $ \alpha = 100 $.

3. Функция Розенброка ($ x \in R^n, \, x^* \equiv 1_n, \, f^* = 0 $):  
		$$
		f(x) = (x_1 - 1)^2 + \alpha \sum_{i=2}^n (x_i - x_{i-1}^2)^2.
		$$  
		Параметр $ \alpha $ можно варьировать. Для тестов возьмем его 10.

4. Задача энтропийно-линейного программирования без ограничений:
		$$
		f(x) = \sum_{i=1}^n x_i \ln \frac{x_i}{\xi_i}, \, x \in R_{+}^n; \, (n = 10, \ldots 1000); \, \xi_i = 1/i
		$$

5. Линейная регрессия:
		$$
		f(x) = \sum_{i=1}^m ((a^i, x) - b_i)^2
		$$  
		Параметр $ m = 100, \, $ значения $ a^i \in R^n, \, b_i $ выбираются случайно.

6. Функция правдоподобия:
		$$
		f(x,y) = -\sum_{i=1}^{k}(x^T a^i + y) + \sum_{i=1}^{m}\ln(1 + \exp(x^T a^i + y))
		$$

* **Комментарии**

1. Начальная точка выбирается случайно на достаточно большом удалении от оптимальной точки. Расстояние фиксируется одним и тем же для разных размерностей задачи.  
2. Точность решения варьируется от $10^{-4}$ до $10^{-5}$ с шагом $10^{-5}$.  
3. Точность одномерного поиска варьируется от $10^{-7}$ до $10^{-8}$ с шагом $10^{-8}$.  
4. Размерность задачи варьируется: 10, 20, 30, 40, 50, 60, 60, 70, 80, 90, 100, 200, 400, 600, 800, 1000.  

* **Графики**

1. Для фиксированных выбранных значений точности одномерного поиска и точности решения задачи по функции построить график зависимости времени решения от размерности задачи.  
2. Для фиксированных выбранных значений размерности задачи и точности одномерного поиска построить график зависимости времени решения задачи от требуемой точности решения задачи по функции.  
3. Для фиксированных выбранных значений размерности и точности решения задачи по функции построить график зависимости времени решения от точности одномерного поиска.  
4. Для фиксированных выбранных значений точности одномерного поиска, размерности задачи и точности решения задачи по функции построить график зависимости времени решения от расстояния между начальной точкой и оптимальной точкой.  

In [6]:
import matplotlib.pyplot as plt
import autograd.numpy as np
from autograd import grad
from autograd import jacobian
import time
import copy
from typing import Callable

In [ ]:
#Eugine
# Метод золотого сечения из прошлого проекта
from typing import Callable
def golden_section_search(f: Callable[[float, float], float], a: float, b: float, d: np.array,  eps: float) -> tuple[float, list, list, int]:
    phi = (1 + np.sqrt(5)) / 2
    x1 = a + (b - a) / (phi + 1)
    x2 = b - (b - a) / (phi + 1)
    f1, f2 = f(x1, d), f(x2, d)

    iter_num, lst_x, lst_y = 0, [], []
    lst_x.append(0)
    lst_y.append((a+b)/2)

    while abs(b - a) > eps:
        iter_num += 1
        if f1 < f2:
            b = x2
            x2, f2 = x1, f1
            x1 = a + (b - a) / (phi + 1)
            f1 = f(x1, d)
        else:
            a = x1
            x1, f1 = x2, f2
            x2 = b - (b - a) / (phi + 1)
            f2 = f(x2, d)
        
        lst_x.append(iter_num)
        lst_y.append((a+b)/2)
    return ((a + b) / 2, copy.deepcopy(lst_x), copy.deepcopy(lst_y), iter_num)


In [ ]:
# Eugine 

eps_fastest = 10**(-4)
eps_golden = 10**(-7)

def numerical_gradient(f: Callable[[np.array], float], x: np.array, eps=1e-6):
    grad = np.zeros_like(x)
    print(type(x))
    for i in range(len(x)):
        x_plus = x.copy()
        x_plus[i] += eps
        x_minus = x.copy()
        x_minus[i] -= eps
        grad[i] = (f(x_plus) - f(x_minus)) / (2 * eps)
    return grad

def the_fastest_descent(f: Callable[[np.array], float], x: np.array, eps: float):
    def func_of_h(h:float, d:float) -> float:
        return f(x + h*d)

    d = -numerical_gradient(f, x)
    while(np.linalg.norm(d)**2 > eps):
        x += (golden_section_search(func_of_h, 0, 100, d, eps_golden)[0])*d 
        d = -numerical_gradient(f, x)

    return x



In [ ]:
# Griborii
def hessian(func, x: np.array):
    return jacobian(grad(func))(x)

def levenberg_markvatd(f: Callable[[np.array], float], x_0: np.array, a_0: float, prec):
    x_old = np.copy(x_0)
    x_new = np.copy(x_0 + 2 * prec)
    a = a_0
    while max(x_old - x_new) < prec:
        x_old = np.copy(x_new)
        print(hessian(f, x_old))
        x_new = np.copy(x_old + a * np.linalg.inv(hessian(f, x_old)) @ grad(f)(x_old))
        print(  x_new, f(x_new), f(x_old))
        while f(x_new) >= f(x_old):
            a /= 2
            x_new = np.copy(x_old + a * np.linalg.inv(hessian(f, x_old)) @ grad(f)(x_old))

    return x_new

pow_2 = lambda x: sum([x[i]**2 for i in range(len(x))])
print(levenberg_markvatd(pow_2, np.array([2.0, 2.0]), 0.05, 0.05))

In [ ]:
# test funcs

# Сильно-выпуклая функция
def strongly_convex(x: np.ndarray, L: int = 100, mu: float = 0.1) -> float:
	return ((L - mu) / 8) * (x[0]**2 + np.sum((x[:-1] - x[1:])**2) - 2 * x[0]) + (mu / 2) * np.linalg.norm(x)**2

# Двойственная задача матрицы корреспонденций
def dual_correspondence(x: np.ndarray, y: np.ndarray, L: np.ndarray, W: np.ndarray, c: np.ndarray) -> float:
	return -np.dot(L, x) - np.dot(W, y) + np.log(np.sum(np.exp(-100 * c + x[:, None] + y[None, :])))

# Функция Розенброка
def rosenbrock(x: np.ndarray, alpha_rosenbrock: int = 10) -> float:
	return (x[0] - 1)**2 + alpha_rosenbrock * np.sum((x[1:] - x[:-1]**2)**2)

# Энтропийно-линейная функция
def entropy_linear(x: np.ndarray, xi: np.ndarray) -> float:
	return np.sum(x * np.log(x / xi))

# Линейная регрессия
def linear_regression(x: np.ndarray, A: np.ndarray, b: np.ndarray) -> float:
	return np.sum((A @ x - b)**2)

# Функция правдоподобия
def likelihood_function(x: np.ndarray, y: float, A: np.ndarray) -> float:
	return -np.sum(A @ x + y) + np.sum(np.log(1 + np.exp(A @ x + y)))

function_list = [strongly_convex, dual_correspondence, rosenbrock, entropy_linear, linear_regression, likelihood_function]

name_list = [
    "StronglyConvex",
    "DualCorrespondence",
    "Rosenbrock",
    "EntropyLinear",
    "LinearRegression",
    "LikelihoodFunction"
]

number_of_func = 6

# Пример для StronglyConvex
x = np.random.randn(5)
print("StronglyConvex:", function_list[0](x))

# Пример для DualCorrespondence
n = 5
L = np.abs(np.random.rand(n))
W = np.abs(np.random.rand(n))
L /= np.sum(L)
W /= np.sum(W)
c = np.random.rand(n, n)
y = np.random.randn(n)
print("DualCorrespondence:", function_list[1](x, y, L, W, c))

# Пример для Rosenbrock
print("Rosenbrock:", function_list[2](x))

# Пример для EntropyLinear
n = 10
x = np.abs(np.random.rand(n))  # Должно быть положительным
xi = 1 / np.arange(1, n + 1)
print("EntropyLinear:", function_list[3](x, xi))

# Пример для LinearRegression
m, n = 100, 10
A = np.random.randn(m, n)
b = np.random.randn(m)
x = np.random.randn(n)
print("LinearRegression:", function_list[4](x, A, b))

# Пример для LikelihoodFunction
k, m = 5, 100
A = np.random.randn(m, k)
x = np.random.randn(k)
y = np.random.randn()
print("LikelihoodFunction:", function_list[5](x, y, A))
